# Cross-Entropy Method
---
In this notebookm we will train a Cross-Entropy Method with OpenAI Gym's MountainCarContinuous environment

## 1. Import the Necessary Packages

In [2]:
import gym 
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt 
%matplotlib inline 


import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

## 2. Instantiate the Environment and Agent

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

env= gym.make('MountainCarContinuous-v0')
env.seed(101)
np.random.seed(101)

print('observation space:', env.observation_space)
print('action space:', env.action_space)
print(' - low:', env.action_space.low)
print(' - high:', env.action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
observation space: Box(2,)
action space: Box(1,)
 - low: [-1.]
 - high: [1.]


In [17]:
class Agent(nn.Module):
    def __init__(self,env,h_size=16):
        super().__init__()
        self.env = env
        # state, hidden layer, action_sizes
        self.s_size = env.observation_space.shape[0]
        self.h_size = h_size
        self.a_size = env.action_space.shape[0]
        ## define layer
        self.fc1 = nn.Linear(self.s_size,self.h_size)
        self.fc2 = nn.Linear(self.h_size,self.a_size)
        
    def set_weights(self,weights):
        s_size = self.s_size
        h_size = self.h_size
        a_size =self.a_size
        
        # separate the weights for each layer 
        fc1_end = (s_size*h_size) + h_size
        ## s_size*h_size is the number of weights between input and hidden layer
        ## h_size is the number of bias weights between input and hidden layer
        ## s_size*h_size + h_size tells us the total number of weights between input_layer and hidden_layer 
        fc1_W = torch.from_numpy(weights[:s_size*h_size].reshape(s_size,h_size))
        fc1_b = torch.from_numpy(weights[s_size*h_size:fc1_end])
        fc2_W = torch.from_numpy(weights[fc1_end:fc1_end+(h_size*a_size)].reshape(h_size, a_size))
        fc2_b = torch.from_numpy(weights[fc1_end+(h_size*a_size):])
        
        ## Set the weights for each layer
        self.fc1.weight.data.copy_(fc1_W.view_as(self.fc1.weight.data))
        self.fc1.bias.data.copy_(fc1_b.view_as(self.fc1.bias.data))
        self.fc2.weight.data.copy_(fc2_W.view_as(self.fc2.weight.data))
        self.fc2.bias.data.copy_(fc2_b.view_as(self.fc2.bias.data))
        
        ## the difference between view and reshape is that in reshape
        ## it changes the shape of original data and view return the the 
        ## data in which is requested but original data shape is same
    def get_weights_dim(self):
        ## plus 1 compensate the shape for bias
        return (self.s_size+1)*self.h_size + (self.h_size+1)*self.a_size
    
    def forward(self,x):
        x  = F.relu(self.fc1(x))
        x = F.tanh(self.fc2(x))
        return x.cpu().data
    
    def evaluate(self, weights, gamma=1.0, max_t = 5000):
        self.set_weights(weights)
        episode_return = 0.0
        state = env.reset()
        for t in range(max_t):
            state = torch.from_numpy(state).float().to(device)
            action = self.forward(state)
            state,reward,done,_ =env.step(action)
            episode_return += reward*math.pow(gamma,t)
            ## elegant implementation discounting the reward
            if done:
                break
        return episode_return
        
agent = Agent(env).to(device)

        
        

## 3. Train the Agent with a Cross-Entropy Method

In [ ]:
def cem(n_iterations=500, max_t= 1000, gamma=1.0, print_every=100, pop_size=50, elite_frac=0.2, sigma=0.5):
    """PyTorch implementation of a cross-entropy method.
    
    Params
    ======
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode 
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise
        
    """
    n_elite = int(pop_size*elite_frac)
    
    scores_deque = deque(maxlen = 100)
    scores= []
    best_weight = sigma*np.random.randn(agent.get_weights_dim())
    
    for i_iteration in range(1, n_iterations+1):
        
        ## In below step we are training one complete episode for pop_size times (e.g 50 episodes and with different
        ## initializing and collecting all cumulative rewards; 50 rewards in this case) 
        weights_pop = [best_weight + (sigma*np.random.randn(agent.get_weights_dim())) for i in range(pop_size)]
        rewards = np.array([agent.evaluate(weights,gamma,max_t) for weights in weights_pop])
        
        ## Now getting the index(weight) of top 20% of model.
        elite_idxs = rewards.argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        best_weight =np.array(elite_weights).mean(axis=0)
        
        reward = agent.evaluate(best_weight, gamma=1.0)
        scores_deque.append(reward)
        scores.append(reward)
        
        torch.save(agent.state_dict(), 'checkpoint.pth')
        
        if i_iteration%print_every==0:
            print('Episode {}\Average Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))
            
        if np.mean(scores_deque)>=90.0:
            print('\nEnvironment solved in {:d} iterations!\tAverage Score: {:.2f}'.fromat(
                i_iteration-100,np.mean(scores_deque)))
            break
    return scores

scores =cem()
        

In [7]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

10.0